In [1]:
import rqdatac
rqdatac.init()

/Users/domi/Python/.conda/lib/python3.11/site-packages/rqdatac/client.py:257: UserWarning: Your account will be expired after  24 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


In [2]:
from WindPy import *
ret = w.start()
print(ret)
ret = w.isconnected()
print(ret)
#test WSD function
ret = w.wsd("000001.SZ", "sec_name", "2022-05-08", "2022-06-08", "")
print(ret)

24.5.0.60024


2025-01-20 23:30:38.939 python[80888:6151242] +[IMKClient subclass]: chose IMKClient_Modern


Wind.Cosmos.Base V1.7 compiled time is Apr 17 2023, BuildType:Release, CPUArch:X64, GCC Version:Apple LLVM 13.0.0 (clang-1300.0.29.30)
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
.ErrorCode=0
.Data=[OK!]
True
.ErrorCode=0
.Codes=[000001.SZ]
.Fields=[SEC_NAME]
.Times=[20220509,20220510,20220511,20220512,20220513,20220516,20220517,20220518,20220519,20220520,...]
.Data=[[平安银行,平安银行,平安银行,平安银行,平安银行,平安银行,平安银行,平安银行,平安银行,平安银行,...]]


In [3]:
'''
【策略含义】
“景气度+资金流+拥挤度”ETF行业轮动策略
自下而上的基于ETF指数成分股信息构建配置策略
景气度为核心维度：主要基于财报以及一致预期数据挖掘行业内在景气价值
资金流和拥挤度是交易型指标
景气度指标的表现更加稳定，故采用“景气度为主，资金面+拥挤度为辅”的策略

【行业主题ETF筛选与权重配置规则】
1.首先，基于景气度模型确定核心仓位，通过每月调仓截面上景气度最高的五个ETF跟踪指数等权分配确定基础仓位
2.其次，根据资金流情况调整持仓权重，资金流指标排在前五位的指数仓位翻倍，后五位仓位降低50%
3.最后，根据拥挤度指标确定最终的指数仓位，拥挤度处于2以上仓位降低50%，3以上仓位清零。拥挤度指标取每月末最后十个交易日的最大拥挤度数值
'''

'\n【策略含义】\n“景气度+资金流+拥挤度”ETF行业轮动策略\n自下而上的基于ETF指数成分股信息构建配置策略\n景气度为核心维度：主要基于财报以及一致预期数据挖掘行业内在景气价值\n资金流和拥挤度是交易型指标\n景气度指标的表现更加稳定，故采用“景气度为主，资金面+拥挤度为辅”的策略\n\n【行业主题ETF筛选与权重配置规则】\n1.首先，基于景气度模型确定核心仓位，通过每月调仓截面上景气度最高的五个ETF跟踪指数等权分配确定基础仓位\n2.其次，根据资金流情况调整持仓权重，资金流指标排在前五位的指数仓位翻倍，后五位仓位降低50%\n3.最后，根据拥挤度指标确定最终的指数仓位，拥挤度处于2以上仓位降低50%，3以上仓位清零。拥挤度指标取每月末最后十个交易日的最大拥挤度数值\n\n'

In [4]:
import pandas as pd
# 设置显示最大行数为None，表示显示所有行
pd.reset_option('display.max_rows', None)
rqdatac.get_industry_mapping(source='citics', date=None, market='cn')

,first_industry_code,first_industry_name,second_industry_code,second_industry_name,third_industry_code,third_industry_name
0,10,石油石化,1010,石油开采Ⅱ,101010,石油开采Ⅲ
1,10,石油石化,1020,石油化工,102010,炼油
2,10,石油石化,1020,石油化工,102040,油品销售及仓储
3,10,石油石化,1020,石油化工,102050,其他石化
4,10,石油石化,1030,油田服务Ⅱ,103010,油田服务Ⅲ
...,...,...,...,...,...,...
273,63,传媒,6330,文化娱乐,633040,其他文化娱乐
274,63,传媒,6340,互联网媒体,634010,信息搜索与聚合
275,63,传媒,6340,互联网媒体,634020,社交与互动媒体
276,63,传媒,6340,互联网媒体,634030,互联网影视音频


In [5]:
'''
1.剔除最新规模在2亿元以下的ETF
2.通过Wind获取所有股票ETF及跟踪指数，再根据跟踪指数成分筛选持有行业集中的ETF指数，计算每个ETF指数在二级子行业的持仓比例，然后逐个行业选择持股比例最高的ETF跟踪指数
3.
'''
all_etf_df = pd.DataFrame(w.wset("sectorconstituent", "date=20250120;sectorId=1000009165000000;").Data).T
all_etf_df.columns = ["查询日期","ETF代码","ETF简称"]
trackcode_info = w.wss(all_etf_df["ETF代码"].tolist(), "fund_trackindexcode,  fund_trackindexname,  val_mv_ARD", "unit=1;tradeDate=20250119;").Data
# 将trackcode_list合并到all_etf_df中
all_etf_df[["跟踪指数代码","跟踪指数简称","跟踪指数市值"]] = pd.DataFrame(trackcode_info).T
# 筛选出市值在2亿以上的ETF
etf_above_2million = all_etf_df[all_etf_df["跟踪指数市值"] > 200000000]

In [7]:
all_trackCode_list = etf_above_2million["跟踪指数代码"].unique().tolist()
trackcode_maxIndustry_dic = {}
trackcode_maxIndustry_weights_dic = {}
import datetime,dateutil
today_date = datetime.date.today
for i in range(len(all_trackCode_list)):
    index_components = pd.DataFrame(w.wset("indexconstituent", "date=20250120;windcode="+all_trackCode_list[i]+";").Data).T
    index_components.columns = ["date","wind_code","sec_name","i_weight","industry"]
    index_components["rq_code"] = index_components["wind_code"].apply(rqdatac.id_convert)
    industry_info = rqdatac.get_instrument_industry(index_components["rq_code"].tolist(),source='citics_2019',level=0)
    # 将行业信息合并到index_components中
    index_components_merged = pd.merge(index_components,industry_info,left_on="rq_code",right_on="order_book_id")
    # 计算index_components_merged中i_weights之和最大的second_industry_name,并输出
    index_2industry_weights = index_components_merged.groupby("second_industry_name")["i_weight"].sum()
    index_2industry_weights = index_2industry_weights.sort_values(ascending=False)
    trackcode_maxIndustry_dic[all_trackCode_list[i]] = index_2industry_weights.index[0]
    trackcode_maxIndustry_weights_dic[all_trackCode_list[i]] = index_2industry_weights[0]

ValueError: unknown order_book_id: 830799

In [ ]:
# 将trackcode_maxIndustry_dic和trackcode_maxIndustry_weights_dic合并到etf_above_2million中
trackcode_maxIndustry_dic
# etf_above_2million["跟踪指数最大比例行业"] = etf_above_2million["跟踪指数代码"].map(trackcode_maxIndustry_dic)
# etf_above_2million["跟踪指数最大比例行业权重"] = etf_above_2million["跟踪指数代码"].map(trackcode_maxIndustry_weights_dic)
# etf_above_2million

In [ ]:
etf_above_2million

In [ ]:
max_weight_industry = index_2industry_weights.index[0]

max_weight_industry

In [ ]:
len(index_2industry_weights)

In [ ]:
rqdatac.get_instrument_industry(index_components["rq_code"].tolist(),source='citics_2019',level=0)
